In [226]:
import numpy as np
# import gym

In [227]:
# Note: this random walk class can eventually be general, if we
# find out how to algorithmically produce features given the desired
# size of the random walk.  For now, I copied features out of the paper
# and so it has to be size 6.
class random_walk:
    
    def __init__(self, size, policy, gamma=.99):
        """ size: int, size of random walk
            policy: function, returns -1 (left) or 1 (right)
            gamma: float between 0 and 1, discounting factor
        """
        self.policy = policy
        self.size = size
        self.pos = int(size / 2) + 1  # start in the middle of the states
        self.gamma = gamma
        self.features = np.array([
            [1,0,0,0,0],
            [1/np.sqrt(2), 1/np.sqrt(2), 0,0,0],
            [1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0, 0],
            [0, 1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), 0],
            [0, 0, 1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)],
            [0,0,0,0,0]
        ]) # these features are coming right out of the paper.  I did not figure out how he came up with them
        
        self.state = self.features[self.pos-1]  # pos describes the position of the walk.  state is the features at that position
    
    
    def step(self):
        """ Take a step according to the policy, and observe new state, reward, and terminality.
            Returns: new state (features), reward, and a bool end which is True if terminal state, False otherwise
        """
        self.pos += self.policy()
#         if self.pos < 1: self.pos = 1       #   In the paper, state 1 was not terminal.  Right now it's terminal with reward 0
        reward = 1 if self.pos == self.size else 0
        
        if self.pos == self.size or self.pos == 1:
            end = True
        else: end = False
            
        self.state = self.features[self.pos-1]
    
        return self.state, reward, end
    
    
    def reset(self):
        """ Reset the random walk, at the end of an episode.
        """
        self.__init__(self.size, self.policy)
        
    
            
        

In [228]:
class td_pred:
    """ td_lambda for prediction (not control, that has to be a separate class)
        with linear function approximation
    """
    
    def __init__(self, task, lam):
        """ Task: hopefully this will be general enough that we can pass in a random walk class, mountain car task,
                    or whatever else, and td_pred can use them all the same way.
            lam: lambda
        """
        self.task = task
        self.weights = np.zeros_like(task.state)  # initialize weights like state features
        self.lam = lam  # lambda in the algorithm
        
    
    def learn(self, n, alpha):
        """ Perform the td algorithm.  This is straight out of the paper
            n: int, number of episodes
            alpha: float from 0 to 1, step size
            
            returns: weights, state value estimates
        """
        
        gamma = self.task.gamma  
        lam = self.lam
        for _ in range(n):
            e = np.zeros_like(self.task.state)
            s = self.task.state
            vs = self.weights.dot(self.task.state)
            
            while(1):
                s_, r, t = self.task.step()
                vs_ = self.weights.dot(s_)
                d = r + self.task.gamma * vs_ - vs
                e = gamma * lam * e + alpha * (1 - gamma * lam * e.dot(s)) * s
                self.weights = self.weights + d * e + alpha * (vs - self.weights.dot(s)) * s
                vs = vs_
                s = s_
                
                if t:
                    self.task.reset()
                    break
        
        return self.weights, self.task.features.dot(self.weights)
                

In [229]:
# make a policy and instantiate random walk task
p = [.5, .5]
pol = lambda : np.random.choice([-1, 1], p=p)
rw = random_walk(6, pol)

In [230]:
# A couple random walk demos

td = td_pred(rw, .5)  # lambda .5
td.learn(1000, .1)  # learning rate .1

(array([ 0.39566899,  0.38965813,  0.40301077,  0.59082063,  0.59683149]),
 array([ 0.39566899,  0.55531014,  0.68608721,  0.79875806,  0.91836965,  0.        ]))

In [234]:
td = td_pred(rw, .5) # lambda .5
td.learn(2000, .01)  # learning rate .01

(array([ 0.28448093,  0.25833193,  0.35337303,  0.5152536 ,  0.5414026 ]),
 array([ 0.28448093,  0.38382666,  0.51741317,  0.65064983,  0.81408075,  0.        ]))

In [235]:
td = td_pred(rw, .2) # lambda .2
td.learn(2000, .1)  # learning rate .1

(array([ 0.41994216,  0.39043767,  0.36121393,  0.55783837,  0.58734286]),
 array([ 0.41994216,  0.57302507,  0.67641997,  0.75603439,  0.86971765,  0.        ]))

In [236]:
td = td_pred(rw, .2) # lambda .2
td.learn(2000, .01)  # learning rate .01

(array([ 0.28378454,  0.2799094 ,  0.35632085,  0.53937073,  0.54324587]),
 array([ 0.28378454,  0.39859181,  0.53117079,  0.67873354,  0.83077092,  0.        ]))